# Extract Visual Data
Status: Implemented
 -- Fully automatized

In [1]:
import warnings

import pandas as pd
from tqdm.notebook import tqdm
from utils.utils import *

warnings.filterwarnings("ignore")

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
hdf_files = get_hdf_files_per_participant()
sequence_orders = get_sequence_order_per_participant()
assert (set(folders.keys()) == set(hdf_files.keys()))
assert (set(folders.keys()) == set(sequence_orders.keys()))
snippets = {p: list(sequence_orders[p][SNIPPET].values)
            for p in sequence_orders}

### Extract gaze data
* convert gaze coordinates
* extract gazes for each trial

In [3]:
# Iterate over all participants
all_excluded_gazes = {}
for participant in tqdm(folders):
    all_excluded_gazes[participant] = {}
    print('----------------------------------------------')
    print(PARTICIPANT, participant)

    event_data_files = hdf_files[participant]
    behavioral_data = pd.read_csv(get_behavioral_data_path(
        participant, final_data_exclusion=True), index_col=False, sep=SEPARATOR, dtype={PARTICIPANT: str})

    participant_snippet_gazes = {}
    participant_fixation_cross_gazes = {}

    for i, hdf_file in tqdm(enumerate(event_data_files)):
        print(f'(01/05) Load visual events from file {hdf_file}')
        eye_events = retrieve_eye_events(hdf_file)
        if eye_events.empty:
            print(f'No visual events found in file {hdf_file}.')
            update_exclusions(participant, HDF_FILE, VISUAL, {
                hdf_file.name: '01d1_PRE_VIS_Event_Gaze_Data: No visual events found in file.'})
            continue

        print('(02/05) Clean and prepare gaze data')
        # set illogical values to nan, transform to pixel coordinates based on resolution of the monitor
        eye_events: pd.DataFrame = clean_prepare_eye_events(eye_events)
        eye_events[PARTICIPANT] = participant
        # display(eye_events)

        print('(03/05) Extract gaze data that is in accordance with each snippet\'s timing data')
        # per snippet from behavioral dataframe, get corresponding eye events
        # column to compare: time from eye-tracking vs. start and end from behavioral_data
        snippet_gazes = extract_relevant_eye_events(
            behavioral_data[behavioral_data[HDF_INDEX] == i], eye_events)
        fixation_cross_gazes = extract_relevant_eye_events(
            behavioral_data[behavioral_data[HDF_INDEX] == i], eye_events, get_fixation_cross=True)

        print('(04/05) Basic check of each snippet\'s gazes')
        # per snippet perform basic check
        excluded_gazes = check_gaze_data(
            snippet_gazes, fixation_cross_gazes, participant, '01d1_PRE_VIS_Event_Gaze_Data')
        all_excluded_gazes[participant] |= excluded_gazes
        participant_snippet_gazes |= snippet_gazes
        participant_fixation_cross_gazes |= fixation_cross_gazes

    print('(05/05) Store gaze data')
    # save gaze data
    for snippet in tqdm(participant_snippet_gazes):
        gaze_data = participant_snippet_gazes[snippet]
        gaze_data.to_csv(get_snippet_gaze_path(
            participant, snippet), index=False, sep=SEPARATOR)
    for snippet in tqdm(participant_fixation_cross_gazes):
        gaze_data = participant_fixation_cross_gazes[snippet]
        gaze_data.to_csv(get_snippet_gaze_path(
            participant, snippet, fixation_cross=True), index=False, sep=SEPARATOR)
print('Excluded trials per participant', all_excluded_gazes)
print('All trials', sum([len(trials)
      for trials in all_excluded_gazes.values()]))

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003


0it [00:00, ?it/s]

(01/05) Load visual events from file ..\08-Data-Trial_Recordings\raw\E003\Experiment_003_part01.hdf5
(02/05) Clean and prepare gaze data
(03/05) Extract gaze data that is in accordance with each snippet's timing data
(04/05) Basic check of each snippet's gazes
(05/05) Store gaze data


  0%|          | 0/72 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

(01/05) Load visual events from file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part01.hdf5
(02/05) Clean and prepare gaze data
(03/05) Extract gaze data that is in accordance with each snippet's timing data
(04/05) Basic check of each snippet's gazes
(01/05) Load visual events from file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part02.hdf5
(02/05) Clean and prepare gaze data
(03/05) Extract gaze data that is in accordance with each snippet's timing data
(04/05) Basic check of each snippet's gazes
(01/05) Load visual events from file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part03.hdf5
(02/05) Clean and prepare gaze data
(03/05) Extract gaze data that is in accordance with each snippet's timing data
(04/05) Basic check of each snippet's gazes
(01/05) Load visual events from file ..\08-Data-Trial_Recordings\raw\E106\Experiment_106_part04.hdf5
(02/05) Clean and prepare gaze data
(03/05) Extract gaze data that is in accordance with each snippet's timing data


  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/72 [00:00<?, ?it/s]

Excluded trials per participant {'003': {'54-obf-v0': ['01d1_PRE_VIS_Event_Gaze_Data: Too many NA gazes'], '60-clean-v0': ['01d1_PRE_VIS_Event_Gaze_Data: Too many NA gazes'], '54-obf-v1': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval']}, '004': {}, '005': {'8-clean-v1': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval']}, '007': {}, '008': {'60-obf-v1': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval']}, '009': {}, '010': {'7-obf-v2': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval'], '60-obf-v2': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval'], '14-clean-v2': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval'], '54-obf-v0': ['01d1_PRE_VIS_Event_Gaze_Data: Too many NA gazes'], '60-obf-v1': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval']}, '011': {'8-clean-v1': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval']}, '012': {}, '013': {'60-obf-v0': ['01d1_PRE_VIS_Event_Gaze_Data: Too long NA interval'], '54-obf-v1': ['01d1_PRE_VIS_Event_Gaze_Data: T